In [1]:
# import libraries
import pandas as pd

In [63]:
# read csv as dataframe
nyc_pop = pd.read_csv('pop_dataframe_old_names.csv')

In [64]:
nyc_pop

,Unnamed: 0,neighbourhood,pop
0,0,Claremont-Bathgate,28149
1,1,Eastchester-Edenwald-Baychester,35422
2,2,Bedford Park-Fordham North,55329
3,3,Belmont,25967
4,4,Bronxdale,34309
...,...,...,...
385,385,Stapleton-Rosebank,26453
386,386,New Dorp-Midland Beach,21896
387,387,Arden Heights,25238
388,388,Great Kills,40720


In [65]:
# format dataframe for compatibility to nyc_uber_price data
nyc_pop.drop(columns='Unnamed: 0', inplace=True)

In [66]:
nyc_pop.rename(columns={'neighbourhood':'name'}, inplace=True)

In [67]:
nyc_pop

,name,pop
0,Claremont-Bathgate,28149
1,Eastchester-Edenwald-Baychester,35422
2,Bedford Park-Fordham North,55329
3,Belmont,25967
4,Bronxdale,34309
...,...,...
385,Stapleton-Rosebank,26453
386,New Dorp-Midland Beach,21896
387,Arden Heights,25238
388,Great Kills,40720


In [68]:
# use explode method to separate neighborhoods into individual rows
nyc_pop['name'] = nyc_pop['name'].str.split('-')

In [69]:
nyc_pop1 = nyc_pop.explode('name')

In [70]:
pop_count = nyc_pop1.groupby(by='pop').count()

In [71]:
# create new formatted dataframe that will contain adjusted population based on number of splits made by explode method
# this is now assuming even distribution between splits
nyc_pop2 = nyc_pop1.merge(pop_count, how='outer', on='pop')

In [72]:
nyc_pop2.rename(columns={'name_x':'name', 'name_y':'pop_count'}, inplace=True)

In [73]:
nyc_pop2['adj_pop'] = nyc_pop2['pop'] / 2

In [74]:
nyc_pop2['adj_pop'] = nyc_pop2['adj_pop'].round()

In [75]:
nyc_pop2

,name,pop,pop_count,adj_pop
0,Claremont,28149,2,14074.0
1,Bathgate,28149,2,14074.0
2,Eastchester,35422,3,17711.0
3,Edenwald,35422,3,17711.0
4,Baychester,35422,3,17711.0
...,...,...,...,...
641,Rosebank,26453,2,13226.0
642,New Dorp,21896,2,10948.0
643,Midland Beach,21896,2,10948.0
644,Arden Heights,25238,1,12619.0


In [76]:
nyc_pop3 = nyc_pop2[['name', 'adj_pop']]

In [77]:
nyc_pop3.sort_values(by='name', inplace=True)

<ipython-input-77-2ac08d4090c6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nyc_pop3.sort_values(by='name', inplace=True)


In [78]:
# function to fully print entire dataframe for visual
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [ ]:
print_full(nyc_upp)

In [322]:
# manually update all discrepancies by manually cross-referencing between uber data names and nyc_geo data names
nyc_upp.at[265, 'adj_pop'] = 7031

In [252]:
# remove columns that do not exist in original nyc_geo data
indexname = nyc_pop3[nyc_pop3['name'] == 'Westchester'].index
nyc_pop3.drop(indexname, inplace=True)

In [254]:
# aggregate and sum by name
nyc_pop3 = nyc_pop3.groupby(by='name').sum()

In [97]:
nyc_pop3.reset_index(inplace=True)

In [257]:
nyc_up = pd.read_csv('nyc_uber_price.csv')

In [259]:
# merge nyc_uber_price data with population data
nyc_upp = nyc_up.merge(nyc_pop3, how='outer', on='name')

In [323]:
# check for null values
nyc_upp.isnull().sum()

name          0
longitude     0
latitude      0
uber_count    0
avg_price     0
adj_pop       0
dtype: int64

In [326]:
# save nyc_uber_price_pop data to csv
nyc_upp.to_csv('nyc_upp_data.csv', index=False)